In [2]:
import torch
import torch.nn as nn
import numpy as np
import math

In [6]:
with open("data/sonar.all-data") as all_data_file:
    lines = all_data_file.readlines()
    all_data = []
    labels = []
    for line in lines:
        line = line.strip().split(',')
        label = line.pop()
        #line = np.asarray(line, dtype=float)
        if label == "R":
            labels.append(0)
            all_data.append(line)
        elif label == "M":
            labels.append(1)
            all_data.append(line)
        else:
            pass
    all_data = np.asarray(all_data, dtype=float)
    labels = np.asarray(labels, dtype=float)


In [ ]:
def normalise_2darray(d2array):
    output_array = []
    for array in d2array:
        x = (array - np.mean(array)) / np.std(array)
        x[x<0] *= -1
        x = (x-np.min(x))/(np.max(x) - np.min(x))
        output_array.append(x)
    return np.asarray(output_array)
x_norm = normalise_2darray(all_data)

In [ ]:
def train_test_split(input_data, input_labels):
    indices = np.random.permutation(input_data.shape[0])
    split_idx = math.floor(input_data.shape[0] * 0.9)
    train_idx, test_idx = indices[:split_idx], indices[split_idx:]
    train_data, test_data = torch.from_numpy(input_data[train_idx,:]).type(torch.FloatTensor), torch.from_numpy(input_data[test_idx,:]).type(torch.FloatTensor)
    train_labels, test_labels = torch.from_numpy(input_labels[train_idx]).type(torch.FloatTensor), torch.from_numpy(input_labels[test_idx]).type(torch.FloatTensor)
    return train_data, test_data, train_labels, test_labels

In [ ]:
train_data, test_data, train_labels, test_labels = train_test_split(x_norm, labels)

In [ ]:
print(all_data)
print(labels)

In [ ]:
class Network(nn.Module):
    def __init__(self, input_size, output_size) -> None:
        super(Network, self).__init__()
        self.input_size = input_size
        self.output_size = output_size
        self.network = self.__setup_network()

    def __setup_network(self):
        net = nn.Sequential(
            nn.Linear(self.input_size, 32),
            nn.ReLU(),
            nn.Linear(32, 32),
            nn.ReLU(),
            nn.Linear(32, 32),
            nn.ReLU(),
            nn.Linear(32, self.output_size),
            nn.Sigmoid(),
        )
        return net
    def forward(self, input):
        return self.network(input)
    
    def fit(self, input, labels, n_epochs, loss_fn, optimizer, batch_size=10):
        for epoch in range(n_epochs):
            start = 0
            #for i in range(math.floor(input.shape[0]/batch_size)):
            #    X_batch = input[start:(start+batch_size)]
            #    Ybatch = labels[start:(start+batch_size)]
            #    start += batch_size
            #    pass
            prediction = self.forward(input)
            loss = loss_fn(prediction, labels)
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
            print(f"Epoch {epoch}:\tloss:{loss}")
            
    


In [ ]:
model = Network(input_size=60, output_size=1)
loss_fn = nn.L1Loss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=0.9, betas=(0.8, 0.95))
model.train()
model.fit(input=train_data, labels=train_labels, n_epochs=100, loss_fn=loss_fn, optimizer=optimizer)

In [ ]:
model.eval()
model.forward(test_data)